In [15]:
'''
    parser = argparse.ArgumentParser(description='Process mp3dir and convert all files in dir to wav')
    parser.add_argument('-m', '--mp3dir',
                       help='mp3Dir directory absolute path where all mp3 for BPM are kept')
    parser.add_argument('-c','--csvFilePath', required=True,
                       help='mp3FileName(column1),bpms(column2) space seprated would be saved here')
    parser.add_argument('--regenerateCSV',
                       help='mp3FileName(column1),bpms(column2) space seprated would be saved here')
    parser.add_argument('-s', '--songToSearch',
                       help='Song for which you need recommendations')

    args = parser.parse_args()
    csvFilePath = args.csvFilePath
    songToSearch = args.songToSearch
    regenerateCSV = args.regenerateCSV
    mp3Dir = args.mp3dir
'''


"\n    parser = argparse.ArgumentParser(description='Process mp3dir and convert all files in dir to wav')\n    parser.add_argument('-m', '--mp3dir',\n                       help='mp3Dir directory absolute path where all mp3 for BPM are kept')\n    parser.add_argument('-c','--csvFilePath', required=True,\n                       help='mp3FileName(column1),bpms(column2) space seprated would be saved here')\n    parser.add_argument('--regenerateCSV',\n                       help='mp3FileName(column1),bpms(column2) space seprated would be saved here')\n    parser.add_argument('-s', '--songToSearch',\n                       help='Song for which you need recommendations')\n\n    args = parser.parse_args()\n    csvFilePath = args.csvFilePath\n    songToSearch = args.songToSearch\n    regenerateCSV = args.regenerateCSV\n    mp3Dir = args.mp3dir\n"

In [16]:
if __name__ == '__main__':
    wavDir = '/home/tivo/Desktop/songs/wav/test'
    mp3Dir = '/home/tivo/Desktop/songs/mp3'
    csvFilePath = '/home/tivo/Desktop/bpms.csv'
    regenerateCSV = True
    
    if (regenerateCSV):
        #wavDir = '/home/tivo/Desktop/songs/wav/test'
        #mp3Dir = '/home/tivo/Desktop/songs/mp3'
        #csvFilePath = '/home/tivo/Desktop/bpms.csv'

        if(mp3Dir is None):
            print ("Error: --mp3dir or -m: where all mp3 are kept is required for regenerate case")
            exit()

        #mp3Dir = args.mp3dir
        bpmE = bpmExtractor()

        for filename in os.listdir(mp3Dir):
            #print (mp3Dir, filename)
            #mp3FilePath = "".join([mp3Dir, "/",filename])
            wavFilePath,mp3FilePath = bpmE.mp3ToWav(mp3Dir,filename)
            print("FileName: {} ".format(wavFilePath))
            if wavFilePath is not None and os.path.exists(wavFilePath):
                bpmE.extractBPMtoDFExisting(wavFilePath,mp3FilePath,5,5)
                #Remove wavFile after use
                os.remove(wavFilePath)
            #print("=="*80)
            #extractBPMtoDF(wavFilePath,10)

        bpmE.bpmsDF.to_csv(csvFilePath)


    songToSearch = '/home/tivo/Desktop/songs/mp3/WGNF.mp3'
    #bpmsDF = pnd.read_csv('/home/tivo/Desktop/bpms_mono_way.csv')
    if(songToSearch is None):
        print ("Error: -s or --songToSearch : aka song for which you need recommendations is required for regenerate case")
        exit()

    #songToSearch = args.songToSearch
    bpmsSaved = pnd.read_csv(csvFilePath)

    sSongsObj = similarSongs(bpmsDF)
    sSongsObj.fit_transform('bpms')
    #sSongsObj.getGenre('/home/tivo/Desktop/songs/mp3/BITE.mp3')

    #songToSearch = '/home/tivo/Desktop/songs/wav/XKPJ.wav'
    similarSongList = sSongsObj.findKSimilarSongs(songToSearch, 5)

    print ("Song requested bpm is {}".format(bpmsDF[bpmsDF['audioFile'] == songToSearch]['bpms']))
    for count,j in enumerate(similarSongList):
        print("Song-{} is: {}".format(count,bpmsDF.loc[j]['audioFile']))
        print("\t bpms is : {}".format(bpmsDF.loc[j]['bpms']))
        print ()



audio/mpeg
FileName: /home/tivo/Desktop/songs/mp3/WGNF.wav 


/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:301: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:127: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:152: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:166: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:177: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/i

application/octet-stream
FileName: /home/tivo/Desktop/songs/mp3/CPGC.wav 


/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:301: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:127: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:152: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:166: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:177: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/i

application/octet-stream
FileName: /home/tivo/Desktop/songs/mp3/XYKV.wav 


...

application/octet-stream
FileName: /home/tivo/Desktop/songs/mp3/WPVO.wav 


/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:301: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:127: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:152: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:166: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:177: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/i

application/octet-stream
FileName: /home/tivo/Desktop/songs/mp3/DMGC.wav 


/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:301: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:127: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:152: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:166: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:177: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/tivo/anaconda3/lib/python3.5/site-packages/i

KeyboardInterrupt: 

In [ ]:
magic.from_file("/home/tivo/Desktop/songs/mp3/DVXM.mp3",mime=True)

In [ ]:
magic.from_file("/home/tivo/Desktop/songs/mp3/HTJH.m4a",mime=True)

In [ ]:
mp3FilePath = "/home/tivo/Desktop/songs/mp3/DVXM.mp3"
f = open(mp3FilePath, "rb")
#print ("[{}]".format(str(f.read(3).decode("utf-8"))))
if str(f.read(3).decode("utf-8")) == 'ID3':
    print("1")
    mp3file = MP3(mp3FilePath, ID3=EasyID3)
    print (mp3file.get("genre"))

In [ ]:
mp3file = MP3(mp3FilePath, ID3=EasyID3)
print (mp3file.get("genre"))

In [ ]:
mp3file.pprint()

In [ ]:
print (mp3file.get("album"))
print (mp3file.get("artist"))
print (mp3file.get("composer"))
print (mp3file.get("title"))



In [22]:
a = [1,2,3]
b = [3,4,5]
c = [7,4,9]
print (numpy.correlate(a,b,'full'))
print (numpy.correlate(a,c,'full'))

[ 5 14 26 18  9]
[ 9 22 42 26 21]
